# Create an Onnx edge configuration package


## Create Preprocessing component

The responsibility of the `Preprocessing Python Component` is to receive `ImageSet payload` and transform it to the input required by the `GPU Runtime Component`.. 

In [ ]:
from simaticai import deployment

preprocessing = deployment.PythonComponent(name='preprocessing', python_version='3.11')

preprocessing.add_resources('../src/preprocessing/', 'preprocessing.py')
preprocessing.set_entrypoint('preprocessing.py')
preprocessing.set_requirements('../src/preprocessing/requirements.txt')

preprocessing.add_input("vision_payload", "ImageSet")

preprocessing.add_output("input", "Float32Array")
preprocessing.add_output("iuid", "String")

preprocessing

## Create Object Detection component

The responsibility of the `Detection GPU Runtime Component` is to produce a prediction from the given input and pass `boxes`, `labels` and `scores`.  

First we add the model to the component, and the listed information will be added automatically.
- model-config.pbtxt
- component inputs required by the model
- component outputs required by the model

In [ ]:
from simaticai.deployment import model_config
detection = deployment.GPURuntimeComponent(name='detection')

detection.use_model("../src/detection/1/model.onnx", 
                    max_batch_size=1, 
                    warmup=model_config.Warmup.RANDOM_DATA)

# adding extra inputs and outputs to connect the payload with the result
detection.add_input("iuid", "String")  
detection.add_output("iuid", "String")

detection

With the code above, AI SDK generates a default model_config for your model, containing the required information as follows
- platform information
- input tensor(s) name, type and shape
- output tensor(s) name, type and shape
- model_warmup

To change this configuration, you can save the component and fine-tune the generated `config.pbtxt` file. 

```python
# python
detection.save("../packages/components/") # saving component
# [...] editing ../packages/components/detection/config.pbtxt file, e.g adding TensorRT optimization
detection.use_config("../packages/components/detection/config.pbtxt")  # using the fine-tuned config file
```

## Create Postprocessing component

The responsibility of the `Postprocessing Python Component` is to receive the predictions from the the `Detection GPU Runtime Component` and make a decision on the results. 

In [ ]:
postprocessing = deployment.PythonComponent(name='postprocessing', python_version='3.11')

postprocessing.add_resources('../src/postprocessing/', 'postprocessing.py')
postprocessing.set_entrypoint('postprocessing.py')
postprocessing.set_requirements('../src/postprocessing/requirements.txt')

postprocessing.add_input("iuid", "String")
postprocessing.add_input("boxes", "Float32Array")
postprocessing.add_input("labels", "Int64Array")
postprocessing.add_input("scores", "Float32Array")

postprocessing.add_output("prediction", "String")
postprocessing.add_output("result", "String")

postprocessing

#### Create a pipeline from this component

Now you can use the component to create a pipeline configuration.  

In [ ]:
#To assure compatibility with older versions of AI SDK (<v1.5.0), you must define the `version` parameter in the `from_components()` method
pipeline = deployment.Pipeline.from_components(
    [preprocessing, detection, postprocessing], 
    name='BoardObjectDetection',
    desc="3 Steps Pipeline for Object model to spot extra or missing holes, not expected damages on the a workpiece")
pipeline

Once the pipeline is created, it can be exported as an edge configuration package and then tested by the notebook [40-TestPipelineLocally](./40-TestPipelineLocally.ipynb).

In [ ]:
edge_package_path = pipeline.export('../packages')
edge_package_path

The edge package can be deployed to the `AI Model Manager` or imported into `AI Inference Server`.